# Рейтинг друзей по jaccard similarity

In [2]:
import collections
import vk
import pandas as pd
import time

In [9]:
VK_TOKEN = 'aa9a438374e5f7808961c523653ba7217c1d8efdba09cfadcdbde9f1a1906e8e970e03a403e119dcbe740'
ID = 10385605
session = vk.Session(VK_TOKEN)
vk_api = vk.API(session, v=5.103)
deq = collections.deque(maxlen=4)

Коэффициент Жаккарда 

In [10]:
def jaccard(l1, l2):
    intersec = len(l1.intersection(l2))
    union = len(l1.union(l2))
    return intersec / union

Метод возвращает список тематик групп пользователя

In [11]:
def get_groups_activity(id):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    groups = vk_api.groups.get(user_id=id, fields=['activity'], extended=1)
    activities = []
    for group in groups['items']:
        try:
            activities.append(group['activity'])
        except Exception:
            continue
    return activities

Возвращает топ друзей по мере Жаккарда

In [12]:
def get_rating():
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    my_activities = get_groups_activity(ID)
    df = pd.DataFrame(columns=['friend_name', 'jaccard'])
    i = 0
    friends = vk_api.friends.get(user_id=ID, extended=1, fields='nickname')['items']
    for friend in friends:
        try:
            activities = get_groups_activity(friend['id'])
            similarity = jaccard(set(my_activities), set(activities))
            friend_name = friend['first_name'] + ' ' + friend['last_name']
            df.loc[i] = [friend_name, similarity]
            i += 1
        except Exception:
            continue
    df = df.sort_values(by='jaccard', ascending=False)
    return df

In [13]:
df = get_rating()
df

,friend_name,jaccard
219,Ксюша Панова,0.365854
90,Азат Нафиков,0.352941
43,Ярослав Петров,0.350515
296,Анастасия Самилова,0.346154
119,Артур Мулихов,0.340426
...,...,...
325,Егор Яровиков,0.015625
281,Гульчачак Айнуллова,0.015385
144,Руслан Бикчентаев,0.000000
3,Alexander Yukhimenko,0.000000
